In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/bernat/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
nltk.download('stopwords')
from nltk.corpus import stopwords
#print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/bernat/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
stop_words = set(stopwords.words('english'))

In [5]:
sentence = 'Once you know all the elements, it’s not difficult to pull together a sentence.'
words = nltk.word_tokenize(sentence)
without_stop_words = [word for word in words if not word in stop_words]

In [6]:
#Читаем датасет из файла
df = pd.read_csv("emails.csv")
df

,text,spam
0,Subject: naturally irresistible your corporate...,1
1,Subject: the stock trading gunslinger fanny i...,1
2,Subject: unbelievable new homes made easy im ...,1
3,Subject: 4 color printing special request add...,1
4,"Subject: do not have money , get software cds ...",1
...,...,...
5723,Subject: re : research and development charges...,0
5724,"Subject: re : receipts from visit jim , than...",0
5725,Subject: re : enron case study update wow ! a...,0
5726,"Subject: re : interest david , please , call...",0


In [7]:
#отсутствуют пропущенные значения
df.isnull().mean()

text    0.0
spam    0.0
dtype: float64

In [54]:
# import libraries
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
import string
from nltk.tokenize import word_tokenize
from sklearn.metrics import confusion_matrix

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.ensemble import AdaBoostClassifier,RandomForestClassifier
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score


import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding,Input,LSTM,Dense,Bidirectional,Dropout, Activation
from keras.models import Model
from tensorflow.keras.models import Sequential
tf.__version__
import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/bernat/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [55]:
df.drop_duplicates(inplace = True)

In [56]:
df['mail_len'] = df.text.apply(len)

In [66]:
vocab_size = 10000
max_len = 250

# Tokenize the mails
tok = Tokenizer(num_words=vocab_size)
tok.fit_on_texts(df.text)

In [82]:
# Use text_to_sequence to convert it into vectors
sequences = tok.texts_to_sequences(df.text)


In [83]:
# pad seqence to create a matrix of equal length mails
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)
sequences_matrix

array([[   0,    0,    0, ...,  693,   40,  215],
       [   0,    0,    0, ..., 3987, 8017,  112],
       [   0,    0,    0, ...,  696,   26,    6],
       ...,
       [  16,   12,    7, ..., 1832,  298,  460],
       [   0,    0,    0, ...,  289,   22,   39],
       [   7,    1,  118, ...,   91,   54,  290]], dtype=int32)

In [58]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(sequences_matrix, df.spam, test_size = 0.2, random_state = 1)

In [59]:
model = RandomForestClassifier()

model_name = 'Random Forest Classifier'
accuracy = 0
clf=model
clf.fit(X_train,y_train)
y_pred=clf.predict(X_test)
accuracy_score(y_test,y_pred)

0.845478489903424

# LSTM

In [52]:
model = Sequential()
model.add(Embedding(vocab_size, 200, input_length = max_len))
model.add(LSTM(32))
model.add(Dense(1,activation='sigmoid'))
model.summary()
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

2022-05-18 15:59:12.628506: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 250, 200)          2000000   
                                                                 
 lstm (LSTM)                 (None, 32)                29824     
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
Total params: 2,029,857
Trainable params: 2,029,857
Non-trainable params: 0
_________________________________________________________________


In [53]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=5,batch_size=64)

Epoch 1/5
72/72 [==============================] - 14s 166ms/step - loss: 0.3408 - accuracy: 0.8624 - val_loss: 0.1647 - val_accuracy: 0.9447
Epoch 2/5
72/72 [==============================] - 11s 154ms/step - loss: 0.0685 - accuracy: 0.9853 - val_loss: 0.0818 - val_accuracy: 0.9728
Epoch 3/5
72/72 [==============================] - 15s 209ms/step - loss: 0.0185 - accuracy: 0.9963 - val_loss: 0.0725 - val_accuracy: 0.9824
Epoch 4/5
72/72 [==============================] - 13s 174ms/step - loss: 0.0072 - accuracy: 0.9993 - val_loss: 0.0650 - val_accuracy: 0.9816
Epoch 5/5
72/72 [==============================] - 13s 179ms/step - loss: 0.0108 - accuracy: 0.9982 - val_loss: 0.0955 - val_accuracy: 0.9693
